#### Import libraries

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import sys
import os
from joblib import dump, load

# Add config to path
sys.path.append('..')
from config.settings import API_KEYS, API_ENDPOINTS, SOLANA_PROTOCOLS, CACHE_CONFIG

print("📊 Solana DeFi Tracker - API Testing")
print(f"Testing {len(SOLANA_PROTOCOLS)} protocols")
print(f"Cache directory: {os.path.abspath('../data')}")

📊 Solana DeFi Tracker - API Testing
Testing 7 protocols
Cache directory: c:\Users\DELL\Documents\Data Science and Analytics\Web3\grp_project\data


#### Create cache directories

In [2]:
os.makedirs('../data/api_responses', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../data/temp', exist_ok=True)
print("✅ Cache directories created")

✅ Cache directories created


#### Helper Functions

In [5]:
def make_request(url, headers=None, params=None, max_retries=3):
    """Make API request with retry logic"""
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt) # Exponential backoff
            else:
                return None

def save_test_data(data, filename, description=""):
    """Save test data with joblib"""
    filepath = f"../data/temp/{filename}"
    dump(data, filepath)
    print(f"💾 Saved: {description} → {filepath}")
    return filepath

#### Test 1: CoinGecko API - Token Prices

In [ ]:
print("🔍 Testing CoinGecko API...")
# Get token IDs for our protocols
token_ids = []
for protocol_key, protocol_info in SOLANA_PROTOCOLS.items():
    if protocol_info['coingecko_id']:
        token_ids.append(protocol_info['coingecko_id'])
print(f"Token IDs to test: {token_ids}")

if API_KEYS['coingecko']:
    headers = {'X-Cg-Pro-Api-Key': API_KEYS['coingecko']}
   
    # Test simple price endpoint
    price_url = f"{API_ENDPOINTS['coingecko']['base_url']}/simple/price"
    params = {
        'ids': ','.join(token_ids),
        'vs_currencies': 'usd',
        'include_market_cap': 'true',
        'include_24hr_vol': 'true',
        'include_24hr_change': 'true'
    }
   
    price_data = make_request(price_url, headers=headers, params=params)
   
    if price_data:
        print("✅ CoinGecko API working!")
        print(f"📈 Got price data for {len(price_data)} tokens")
       
        # Display sample data
        for token, data in list(price_data.items())[:3]:
            print(f" {token}: ${data.get('usd', 'N/A')} (MCap: ${data.get('usd_market_cap', 'N/A'):,})")
       
        # Save to cache
        save_test_data(price_data, 'coingecko_prices_test.pkl', 'CoinGecko price data')
    else:
        print("❌ CoinGecko API failed")
else:
    print("⚠️ CoinGecko API key not found in .env")

🔍 Testing CoinGecko API...
Token IDs to test: ['jupiter-exchange-solana', 'jito-governance-token', 'meteora', 'raydium', 'marinade', 'kamino', 'orca']
✅ CoinGecko API working!
📈 Got price data for 6 tokens
 jito-governance-token: $2.0 (MCap: $741,526,531.3272717)
 jupiter-exchange-solana: $0.517555 (MCap: $1,581,474,625.0725527)
 kamino: $0.055762 (MCap: $148,265,539.74575433)
💾 Saved: CoinGecko price data → ../data/temp/coingecko_prices_test.pkl


#### Test 2: DefiLlama API - Protocol Data

In [ ]:
print("\n🔍 Testing DefiLlama API...")
# DefiLlama is free, no API key needed
base_url = API_ENDPOINTS['defillama']['base_url']
# Test 1: Get all protocols to find our Solana ones
protocols_url = f"{base_url}/protocols"
all_protocols = make_request(protocols_url)
if all_protocols:
    print(f"✅ DefiLlama API working! Found {len(all_protocols)} total protocols")
   
    # Find our Solana protocols
    solana_protocols_found = []
    for protocol in all_protocols:
        protocol_slug = protocol.get('slug', '').lower()
        protocol_name = protocol.get('name', '').lower()
       
        # Check if it matches our target protocols
        for target_slug in [p['defillama_slug'] for p in SOLANA_PROTOCOLS.values() if p['defillama_slug']]:
            if target_slug.lower() in protocol_slug or target_slug.lower() in protocol_name:
                solana_protocols_found.append({
                    'name': protocol['name'],
                    'slug': protocol['slug'],
                    'tvl': protocol.get('tvl', 0),  # Use 0 as fallback if tvl is None
                    'chain': protocol.get('chain', 'Unknown'),
                    'category': protocol.get('category', 'Unknown')
                })
                break
   
    print(f"📊 Found {len(solana_protocols_found)} target protocols:")
    for p in solana_protocols_found:
        tvl_value = p['tvl'] if p['tvl'] is not None else 0  
        print(f" • {p['name']} (${tvl_value:,.0f} TVL)")
   
    # Save protocol discovery data
    save_test_data(solana_protocols_found, 'defillama_protocols_found.pkl', 'DefiLlama protocol discovery')
   
else:
    print("❌ DefiLlama API failed")


🔍 Testing DefiLlama API...
✅ DefiLlama API working! Found 6341 total protocols
📊 Found 26 target protocols:
 • Jito Liquid Staking ($3,172,438,576 TVL)
 • Kamino Lend ($2,731,327,128 TVL)
 • Raydium AMM ($2,305,489,692 TVL)
 • Jupiter Perpetual Exchange ($2,301,128,421 TVL)
 • Jupiter Staked SOL ($1,209,086,508 TVL)
 • Meteora DLMM ($668,238,143 TVL)
 • Orca DEX ($402,251,500 TVL)
 • Jupiter Lend ($355,469,100 TVL)
 • Kamino Liquidity ($291,940,105 TVL)
 • Jito Restaking ($201,019,425 TVL)
 • Meteora vaults ($136,101,191 TVL)
 • Meteora DAMM V1 ($68,057,728 TVL)
 • MojitoSwap ($650,926 TVL)
 • Kujira Orca ($146,402 TVL)
 • Meteora Dynamic Bonding Curve ($27,890 TVL)
 • OrcaDAO ($13,972 TVL)
 • Jupiter Swap ($95 TVL)
 • Jupiter Aggregator ($0 TVL)
 • Ape Jupiter ($0 TVL)
 • Raydium Perps ($0 TVL)
 • Jupiter DCA ($0 TVL)
 • Jito MEV Tips ($0 TVL)
 • Meteora DAMM V2 ($0 TVL)
 • Jupiter Studio ($0 TVL)
 • Orca Wavebreak ($0 TVL)
 • Jito DAO ($0 TVL)
💾 Saved: DefiLlama protocol discovery →

#### Test 3: DefiLlama Fees Data

In [8]:
print("\n🔍 Testing DefiLlama Fees API...")
fees_url = f"{base_url}/overview/fees"
fees_data = make_request(fees_url)
if fees_data:
    print("✅ DefiLlama Fees API working!")
   
    # Look for our protocols in fees data
    our_protocol_fees = {}
   
    if 'protocols' in fees_data:
        for protocol in fees_data['protocols']:
            protocol_name = protocol.get('name', '').lower()
           
            # Check if it's one of our target protocols
            for target_key, target_info in SOLANA_PROTOCOLS.items():
                if target_info['name'].lower() in protocol_name:
                    our_protocol_fees[target_key] = {
                        'name': protocol['name'],
                        'total_24h': protocol.get('total24h', 0),
                        'total_7d': protocol.get('total7d', 0),
                        'revenue_24h': protocol.get('revenue24h', 0),
                        'chain': protocol.get('chain', 'Unknown')
                    }
                    break
   
    print(f"💰 Found fee data for {len(our_protocol_fees)} protocols:")
    for key, data in our_protocol_fees.items():
        print(f" • {data['name']}: ${data['total_24h']:,.0f} (24h fees)")
   
    # Save fees data
    save_test_data(our_protocol_fees, 'defillama_fees_test.pkl', 'DefiLlama fees data')
   
else:
    print("❌ DefiLlama Fees API failed")


🔍 Testing DefiLlama Fees API...
✅ DefiLlama Fees API working!
💰 Found fee data for 5 protocols:
 • Raydium AMM: $680,371 (24h fees)
 • Orca DEX: $475,465 (24h fees)
 • Meteora Dynamic Bonding Curve: $266,126 (24h fees)
 • Jito DAO: $60,230 (24h fees)
 • Jupiter Studio: $9,038 (24h fees)
💾 Saved: DefiLlama fees data → ../data/temp/defillama_fees_test.pkl


#### Test 4: Data Exploration and Validation

In [10]:
print("\n🔍 Data Quality Check...")
# Load and examine saved test data
test_files = [
    ('coingecko_prices_test.pkl', 'CoinGecko Prices'),
    ('defillama_protocols_found.pkl', 'DefiLlama Protocols'),
    ('defillama_fees_test.pkl', 'DefiLlama Fees')
]
data_summary = {}
for filename, description in test_files:
    filepath = f"../data/temp/{filename}"
    try:
        data = load(filepath)
        data_summary[description] = {
            'loaded': True,
            'type': type(data).__name__,
            'size': len(data) if hasattr(data, '__len__') else 'N/A',
            'sample_keys': list(data.keys())[:5] if isinstance(data, dict) else 'N/A'
        }
        print(f"✅ {description}: {data_summary[description]['type']} with {data_summary[description]['size']} items")
    except FileNotFoundError:
        data_summary[description] = {'loaded': False, 'error': 'File not found'}
        print(f"❌ {description}: Not found")
    except Exception as e:
        data_summary[description] = {'loaded': False, 'error': str(e)}
        print(f"❌ {description}: Error - {e}")


🔍 Data Quality Check...
✅ CoinGecko Prices: dict with 6 items
✅ DefiLlama Protocols: list with 26 items
✅ DefiLlama Fees: dict with 5 items


#### Test 5: Protocol Coverage Analysis

In [11]:
print("\n📋 Protocol Coverage Analysis...")
print("Target Protocols vs Available Data:")
print("-" * 50)
for protocol_key, protocol_info in SOLANA_PROTOCOLS.items():
    print(f"\n🏛️ {protocol_info['name']} ({protocol_key})")
    print(f" Category: {protocol_info['category']}")
    print(f" Token: {protocol_info['token_symbol']}")
   
    # Check data availability
    availability = {
        'coingecko_price': '✅' if protocol_info['coingecko_id'] else '❌',
        'defillama_tvl': '✅' if protocol_info['defillama_slug'] else '❌',
        'native_api': '✅' if protocol_info['has_native_api'] else '❌'
    }
   
    for source, status in availability.items():
        print(f" {source}: {status}")


📋 Protocol Coverage Analysis...
Target Protocols vs Available Data:
--------------------------------------------------

🏛️ Jupiter (jupiter)
 Category: DEX Aggregator
 Token: JUP
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ✅

🏛️ Jito (jito)
 Category: Liquid Staking
 Token: JTO
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ✅

🏛️ Meteora (meteora)
 Category: DEX/AMM
 Token: MET
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ❌

🏛️ Raydium (raydium)
 Category: DEX/AMM
 Token: RAY
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ❌

🏛️ Marinade Finance (marinade)
 Category: Liquid Staking
 Token: MNDE
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ❌

🏛️ Kamino Finance (kamino)
 Category: Lending
 Token: KMNO
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ❌

🏛️ Orca (orca)
 Category: DEX/AMM
 Token: ORCA
 coingecko_price: ✅
 defillama_tvl: ✅
 native_api: ❌


#### Test 6: Sample Data Processing 

In [14]:
print("\n⚙️ Sample Data Processing...")
# Create a sample dataset for testing calculations
try:
    # Load CoinGecko price data
    price_data = load('../data/temp/coingecko_prices_test.pkl')
    fees_data = load('../data/temp/defillama_fees_test.pkl')
   
    # Create sample metrics DataFrame
    sample_metrics = []
   
    for protocol_key, protocol_info in SOLANA_PROTOCOLS.items():
        if protocol_info['coingecko_id'] and protocol_info['coingecko_id'] in price_data:
            token_data = price_data[protocol_info['coingecko_id']]
           
            # Basic metrics
            metrics = {
                'protocol': protocol_info['name'],
                'symbol': protocol_info['token_symbol'],
                'category': protocol_info['category'],
                'price_usd': token_data.get('usd', 0),
                'market_cap': token_data.get('usd_market_cap', 0),
                'volume_24h': token_data.get('usd_24h_vol', 0),
                'price_change_24h': token_data.get('usd_24h_change', 0),
                'fees_24h': 0, # Will be populated from DefiLlama
                'timestamp': datetime.now()
            }
           
            # Add fee data if available
            if protocol_key in fees_data:
                metrics['fees_24h'] = fees_data[protocol_key]['total_24h']
                metrics['revenue_24h'] = fees_data[protocol_key]['revenue_24h']
           
            sample_metrics.append(metrics)
   
    # Create DataFrame
    df_metrics = pd.DataFrame(sample_metrics)
   
    if not df_metrics.empty:
        print("✅ Sample metrics DataFrame created!")
        print(f"📊 Shape: {df_metrics.shape}")
        print("\nSample data:")
        print(df_metrics[['protocol', 'category', 'symbol', 'price_usd', 'market_cap', 'fees_24h']])
       
        # Save processed sample data
        save_test_data(df_metrics, 'sample_metrics.pkl', 'Sample metrics DataFrame')
       
        # Calculate sample P/F ratios
        df_metrics['pf_ratio'] = df_metrics['market_cap'] / (df_metrics['fees_24h'] * 365)
        df_metrics['pf_ratio'] = df_metrics['pf_ratio'].replace([np.inf, -np.inf], np.nan)
       
        print("\n💡 Sample P/F Ratios:")
        pf_data = df_metrics[['protocol', 'pf_ratio']].dropna()
        for _, row in pf_data.iterrows():
            if row['pf_ratio'] < 1000: # Filter out extreme values
                print(f" {row['protocol']}: {row['pf_ratio']:.1f}x")
   
    else:
        print("⚠️ No matching data found for metrics calculation")
except FileNotFoundError:
    print("⚠️ Test data files not found. Run API tests first.")
except Exception as e:
    print(f"❌ Error processing sample data: {e}")


⚙️ Sample Data Processing...
✅ Sample metrics DataFrame created!
📊 Shape: (6, 10)

Sample data:
           protocol        category symbol  price_usd    market_cap  fees_24h
0           Jupiter  DEX Aggregator    JUP   0.517555  1.581475e+09      9038
1              Jito  Liquid Staking    JTO   2.000000  7.415265e+08     60230
2           Raydium         DEX/AMM    RAY   3.690000  9.915826e+08    680371
3  Marinade Finance  Liquid Staking   MNDE   0.127832  7.174761e+07         0
4    Kamino Finance         Lending   KMNO   0.055762  1.482655e+08         0
5              Orca         DEX/AMM   ORCA   2.320000  1.391658e+08    475465
💾 Saved: Sample metrics DataFrame → ../data/temp/sample_metrics.pkl

💡 Sample P/F Ratios:
 Jupiter: 479.4x
 Jito: 33.7x
 Raydium: 4.0x
 Orca: 0.8x


In [1]:
import requests
import json
from datetime import datetime

def get_solana_protocol_fees():
    """
    Fetch fees and revenue data for all Solana-based protocols from DeFiLlama API
    """
    try:
        # API endpoint for Solana chain fees/revenue data
        url = "https://api.llama.fi/overview/fees/solana"
        
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        # Parse JSON response
        data = response.json()
        
        print(f"=== Solana Protocol Fees & Revenue Data ===")
        print(f"Retrieved at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total protocols found: {len(data.get('protocols', []))}\n")
        
        # Extract and display protocol data
        protocols = data.get('protocols', [])
        
        if not protocols:
            print("No protocol data found.")
            return
        
        # Sort protocols by total24h (daily fees) in descending order
        sorted_protocols = sorted(protocols, 
                                key=lambda x: x.get('total24h', 0) or 0, 
                                reverse=True)
        
        print(f"{'Protocol':<25} {'24h Fees':<15} {'24h Revenue':<15} {'7d Fees':<15} {'30d Fees':<15}")
        print("=" * 90)
        
        for protocol in sorted_protocols:
            name = protocol.get('name', 'Unknown')[:24]
            fees_24h = format_currency(protocol.get('total24h'))
            revenue_24h = format_currency(protocol.get('revenue24h'))
            fees_7d = format_currency(protocol.get('total7d'))
            fees_30d = format_currency(protocol.get('total30d'))
            
            print(f"{name:<25} {fees_24h:<15} {revenue_24h:<15} {fees_7d:<15} {fees_30d:<15}")
        
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return None

def format_currency(amount):
    """
    Format currency amount with appropriate units (K, M, B)
    """
    if amount is None or amount == 0:
        return "$0"
    
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.2f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.2f}K"
    else:
        return f"${amount:.2f}"

def get_detailed_protocol_info(data, protocol_name):
    """
    Get detailed information for a specific protocol
    """
    if not data or 'protocols' not in data:
        return None
    
    for protocol in data['protocols']:
        if protocol.get('name', '').lower() == protocol_name.lower():
            print(f"\n=== Detailed Info for {protocol.get('name')} ===")
            print(f"Module: {protocol.get('module', 'N/A')}")
            print(f"Category: {protocol.get('category', 'N/A')}")
            print(f"24h Fees: {format_currency(protocol.get('total24h'))}")
            print(f"24h Revenue: {format_currency(protocol.get('revenue24h'))}")
            print(f"7d Fees: {format_currency(protocol.get('total7d'))}")
            print(f"30d Fees: {format_currency(protocol.get('total30d'))}")
            print(f"Total Fees All Time: {format_currency(protocol.get('totalAllTime'))}")
            return protocol
    
    print(f"Protocol '{protocol_name}' not found.")
    return None

# Main execution
if __name__ == "__main__":
    # Fetch Solana protocol data
    solana_data = get_solana_protocol_fees()
    
    # Optional: Get detailed info for a specific protocol
    # Uncomment the line below and specify a protocol name
    # get_detailed_protocol_info(solana_data, "Jupiter")

=== Solana Protocol Fees & Revenue Data ===
Retrieved at: 2025-08-29 23:07:42
Total protocols found: 116

Protocol                  24h Fees        24h Revenue     7d Fees         30d Fees       
Jupiter Perpetual Exchan  $3.51M          $0              $25.18M         $91.25M        
pump.fun                  $2.07M          $0              $11.54M         $37.74M        
Axiom                     $1.90M          $0              $11.81M         $53.42M        
Solana                    $1.54M          $0              $9.97M          $41.71M        
Meteora DLMM              $1.40M          $0              $8.96M          $55.22M        
PumpSwap                  $1.29M          $0              $7.55M          $27.65M        
Jito MEV Tips             $996.25K        $0              $6.88M          $35.68M        
Raydium AMM               $866.61K        $0              $4.65M          $27.98M        
Orca DEX                  $781.79K        $0              $3.47M          $11.72M   

In [ ]:
import requests
import json
from datetime import datetime

def get_solana_protocol_data(data_type='dailyRevenue'):
    """
    Fetch protocol data for all Solana-based protocols using dataType parameter
    
    Args:
        data_type (str): 'dailyRevenue', 'dailyFees', or 'dailyHoldersRevenue'
    """
    try:
        # API endpoint for Solana chain data
        url = "https://api.llama.fi/overview/fees/solana"
        
        # Parameters to get specific data type
        params = {
            'dataType': data_type,
            'excludeTotalDataChart': 'true',
            'excludeTotalDataChartBreakdown': 'true'
        }
        
        # Make the API request
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Parse JSON response
        data = response.json()
        
        print(f"=== Solana Protocol {data_type.upper()} Data ===")
        print(f"Retrieved at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Total protocols found: {len(data.get('protocols', []))}\n")
        
        # Extract and display protocol data
        protocols = data.get('protocols', [])
        
        if not protocols:
            print("No protocol data found.")
            return None
        
        # Sort protocols by total24h in descending order
        sorted_protocols = sorted(protocols, 
                                key=lambda x: x.get('total24h', 0) or 0, 
                                reverse=True)
        
        # Display data
        data_label = data_type.replace('daily', '').replace('Revenue', ' Revenue').replace('Fees', ' Fees')
        print(f"{'Protocol':<25} {'24h' + data_label:<15} {'7d' + data_label:<15} {'30d' + data_label:<15} {'All Time':<15}")
        print("=" * 95)
        
        protocols_with_data = 0
        
        for protocol in sorted_protocols:
            name = protocol.get('name', 'Unknown')[:24]
            value_24h = format_currency(protocol.get('total24h'))
            value_7d = format_currency(protocol.get('total7d'))
            value_30d = format_currency(protocol.get('total30d'))
            value_all = format_currency(protocol.get('totalAllTime'))
            
            # Count protocols with data
            if protocol.get('total24h', 0) > 0:
                protocols_with_data += 1
            
            print(f"{name:<25} {value_24h:<15} {value_7d:<15} {value_30d:<15} {value_all:<15}")
        
        print(f"\nProtocols with {data_type} > 0: {protocols_with_data}/{len(protocols)}")
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {data_type} data: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON response: {e}")
        return None

def compare_fees_vs_revenue():
    """
    Compare fees data vs revenue data for top protocols
    """
    print(f"\n{'='*85}")
    print("COMPARISON: FEES vs REVENUE for Top 15 Solana Protocols")
    print(f"{'='*85}")
    
    try:
        # Get fees data
        fees_response = requests.get("https://api.llama.fi/overview/fees/solana", 
                                   params={'dataType': 'dailyFees', 'excludeTotalDataChart': 'true'})
        fees_data = fees_response.json()
        
        # Get revenue data  
        revenue_response = requests.get("https://api.llama.fi/overview/fees/solana",
                                      params={'dataType': 'dailyRevenue', 'excludeTotalDataChart': 'true'})
        revenue_data = revenue_response.json()
        
        # Create lookup for revenue data
        revenue_lookup = {p['name']: p for p in revenue_data.get('protocols', [])}
        
        # Sort by fees and take top 15
        top_protocols = sorted(fees_data.get('protocols', []), 
                             key=lambda x: x.get('total24h', 0) or 0, 
                             reverse=True)[:15]
        
        print(f"{'Protocol':<25} {'24h Fees':<15} {'24h Revenue':<15} {'Rev/Fee %':<12}")
        print("-" * 85)
        
        total_fees = 0
        total_revenue = 0
        
        for protocol in top_protocols:
            name = protocol.get('name', 'Unknown')
            fees_24h = protocol.get('total24h', 0)
            
            # Get corresponding revenue data
            revenue_protocol = revenue_lookup.get(name, {})
            revenue_24h = revenue_protocol.get('total24h', 0)
            
            # Calculate percentage
            rev_percentage = (revenue_24h / fees_24h * 100) if fees_24h > 0 else 0
            
            total_fees += fees_24h
            total_revenue += revenue_24h
            
            print(f"{name[:24]:<25} {format_currency(fees_24h):<15} {format_currency(revenue_24h):<15} {rev_percentage:.1f}%")
        
        # Show totals
        overall_percentage = (total_revenue / total_fees * 100) if total_fees > 0 else 0
        print("-" * 85)
        print(f"{'TOTAL (Top 15)':<25} {format_currency(total_fees):<15} {format_currency(total_revenue):<15} {overall_percentage:.1f}%")
        
    except Exception as e:
        print(f"Error in comparison: {e}")

def format_currency(amount):
    """
    Format currency amount with appropriate units (K, M, B)
    """
    if amount is None or amount == 0:
        return "$0"
    
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.2f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.2f}K"
    else:
        return f"${amount:.2f}"

# Main execution
if __name__ == "__main__":
    # Get Solana protocol revenue data
    print("📊 Getting Solana Protocol Revenue Data...")
    get_solana_protocol_data('dailyRevenue')
    
    print("\n" + "="*50 + "\n")
    
    # Get Solana protocol fees data
    print("💰 Getting Solana Protocol Fees Data...")
    get_solana_protocol_data('dailyFees')
    
    # Compare fees vs revenue
    compare_fees_vs_revenue()
    
    print(f"\n{'='*85}")
    print("✅ Available Data Types:")
    print("   - 'dailyRevenue' - Revenue earned by protocols")
    print("   - 'dailyFees' - Total fees paid by users") 
    print(f"{'='*85}")

📊 Getting Solana Protocol Revenue Data...
=== Solana Protocol DAILYREVENUE Data ===
Retrieved at: 2025-08-29 23:48:26
Total protocols found: 103

Protocol                  24h Revenue     7d Revenue      30d Revenue     All Time       
pump.fun                  $2.07M          $11.54M         $37.74M         $747.40M       
Axiom                     $1.90M          $11.81M         $53.42M         $258.38M       
Jupiter Perpetual Exchan  $877.60K        $6.29M          $22.81M         $229.64M       
Phantom Wallet            $606.67K        $3.84M          $16.72M         $437.32M       
PumpSwap                  $216.00K        $1.26M          $4.64M          $26.91M        
Photon                    $177.57K        $1.18M          $6.24M          $423.61M       
Solana                    $157.61K        $1.10M          $4.49M          $553.08M       
GMGN                      $130.34K        $736.14K        $3.38M          $85.81M        
Raydium AMM               $128.90K        $6

In [1]:
import requests
import pandas as pd

# Fetch Jupiter token list
url = "https://token.jup.ag/all"
resp = requests.get(url)
tokens = resp.json()

# Convert to DataFrame
df = pd.DataFrame(tokens)

# Select useful columns
df = df[["address", "symbol", "name", "decimals", "logoURI"]]

print(df.head())


                                        address symbol         name  decimals  \
0   So11111111111111111111111111111111111111112    SOL  Wrapped SOL         9   
1  EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v   USDC     USD Coin         6   
2  Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB   USDT         USDT         6   
3  2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo  PYUSD   PayPal USD         6   
4  EKpQGSJtjMFqKZ9KQanSqYXRcF8fBopzLHYxdM65zcjm   $WIF    dogwifhat         6   

                                             logoURI  
0  https://raw.githubusercontent.com/solana-labs/...  
1  https://raw.githubusercontent.com/solana-labs/...  
2  https://raw.githubusercontent.com/solana-labs/...  
3  https://424565.fs1.hubspotusercontent-na1.net/...  
4  https://bafkreibk3covs5ltyqxa272uodhculbr6kea6...  


In [3]:
df.shape


(287863, 5)